# Notebook de nettoyage des données du dataset X

**Objectifs :**

- Nettoyer les données pour obtenir un seul fichier contenant l'échantillon qui sera utilisée.
- Vérifier la présence de valeurs manquantes ou erronées.
- S'assurer de réunir uniquement les données nécessaires pour la base de données.
- Vérifier le typage des données correspondant à la base de données prévue.

In [103]:
from rich import print
from tqdm import tqdm
import pandas as pd
import glob
import os

### Récupération de la liste des fichiers du dataset disponible dans le dossier origin

Pour avoir le dataset complet prêt pour le traitement :
```bash
$ mkdir origin
$ git clone https://github.com/sinking8/usc-x-24-us-election.git origin
```

In [ ]:
def get_csv_list(part_number):
    csv_files = []
    if part_number is list:
        for part in part_number:
            csv_files += glob.glob(f"./origin/part_{part}/*.csv")
    else:
        csv_files = glob.glob(f"./origin/part_{part_number}/*.csv")
    return csv_files

### Création des DataFrames finaux vides, avec typage

In [ ]:
x_post_df = pd.DataFrame(
    {
        "post_id": pd.Series(dtype="int"),
        "user_id": pd.Series(dtype="int"),
        "lang": pd.Series(dtype="str"),
        "text": pd.Series(dtype="str"),
        "date": pd.Series(dtype="datetime64[ns]"),
        "like_count": pd.Series(dtype="int"),
        "reply_count": pd.Series(dtype="int"),
        "retweet_count": pd.Series(dtype="int"),
        "quote_count": pd.Series(dtype="int"),
    }
)

print(x_post_df.dtypes)
x_post_df.to_parquet("./data/x_post.parquet", index=False)

x_user_df = pd.DataFrame(
    {
        "user_id": pd.Series(dtype="int"),
        "name": pd.Series(dtype="str"),
        "bio": pd.Series(dtype="str"),
        "followers_count": pd.Series(dtype="int"),
        "follows_count": pd.Series(dtype="int"),
    }
)

print(x_user_df.dtypes)
x_user_df.to_parquet("./data/x_user.parquet", index=False)

post_id                   int64
user_id                   int64
lang                     object
text                     object
date             datetime64[ns]
like_count                int64
reply_count               int64
retweet_count             int64
quote_count               int64
dtype: object

user_id             int64
name               object
bio                object
followers_count     int64
follows_count       int64
dtype: object

### Fonction de nettoyage

- entrée : chemin du csv a ajouter, les dataframe originaux
- sortie : les dataframe avec les nouvelles lignes ajoutées

In [117]:
def clean_file(path, x_post_df, x_user_df):
    current_df = pd.read_csv(path)
    for i, row in tqdm(
        current_df.iterrows(),
        total=len(current_df),
        desc=f"Processing {os.path.basename(path)}",
    ):
        try:
            # Vérification des données
            # Nous voulons :
            # - Les lignes sans valeurs nulles
            # - Les tweets en français
            # - Sans retweets ni citation de tweets.
            required_columns = [
                "id",
                "lang",
                "text",
                "date",
                "likeCount",
                "replyCount",
                "retweetCount",
                "quoteCount",
                "user",
            ]
            if any(col not in row or pd.isna(row[col]) for col in required_columns):
                continue
            if row["lang"] != "fr":  # Choisir language voulu
                continue
            if row["quotedTweet"]:
                continue
            if row["retweetedTweet"]:
                continue

            ######### USER PART #########
            import datetime  # Import datetime pour évaluer directement le dictionnaire "user"

            user_dict = eval(row["user"])  # Evaluer la ligne
            user_id = int(user_dict["id"])
            user_name = user_dict["username"]
            user_bio = user_dict["rawDescription"]
            user_followers_count = int(user_dict["followersCount"])
            user_follows_count = int(user_dict["friendsCount"])

            ######### POST PART #########

            post_id = int(row["id"])
            lang = str(row["lang"])
            text = str(row["text"])
            date = str(row["date"])
            like_count = int(row["likeCount"])
            reply_count = int(row["replyCount"])
            retweet_count = int(row["retweetCount"])
            quote_count = int(row["quoteCount"])

            ######### AJOUT PART #########

            # Vérification si le tweet existe déjà dans le DataFrame
            if x_post_df["post_id"].eq(post_id).any():
                continue
            else:
                new_row_tweet = pd.DataFrame(
                    {
                        "post_id": [post_id],
                        "user_id": [user_id],
                        "lang": [lang],
                        "text": [text],
                        "date": [pd.to_datetime(date)],
                        "like_count": [like_count],
                        "reply_count": [reply_count],
                        "retweet_count": [retweet_count],
                        "quote_count": [quote_count],
                    }
                )
                x_post_df = pd.concat([x_post_df, new_row_tweet], ignore_index=True)

            # Vérification si l'utilisateur existe déjà dans le DataFrame
            if x_user_df["user_id"].eq(user_id).any():
                continue
            else:
                new_row_user = pd.DataFrame(
                    {
                        "user_id": [user_id],
                        "name": [user_name],
                        "bio": [user_bio],
                        "followers_count": [user_followers_count],
                        "follows_count": [user_follows_count],
                    }
                )
                x_user_df = pd.concat([x_user_df, new_row_user], ignore_index=True)

        except Exception as e:
            print(
                f"Error processing row {i} from file {os.path.basename(path)}: {str(e)}"
            )
            continue

    return (x_post_df, x_user_df)

### Cellule principale de traitement progressif des données 

Le jeu de données a, depuis sa première publication, reçu des mises à jour. Ces changements ont corrigés des erreurs. Mais les mises à jour ont aussi intégré de nouvelles périodes, couvrant jusqu'à la fin de 2024. 

- Pour la version à date de publication mettre `[i for i in range(1, 23)]` et avoir le git du dataset complet dans le dossier racine.
- Pour une autre sélection, remplacer la variables PARTIES par une liste de numéro de fichier à traiter.

In [ ]:
x_post_df = pd.read_parquet("./data/x_post.parquet")
x_user_df = pd.read_parquet("./data/x_user.parquet")

PARTIES = [
    1 # Pour tester la partie 1 intégrée dans le git
    ]

for path in get_csv_list(
    PARTIES
):
    x_post_df, x_user_df = clean_file(path, x_post_df, x_user_df)

# x_post confirmation
print("[bold yellow]x_post_df describe:[/bold yellow]")
print(x_post_df.describe())
print(x_post_df.shape)
print("[bold yellow]x_post_df head(5):[/bold yellow]")
print(x_post_df.head())
x_post_df.to_parquet("./data/x_post.parquet", index=False)
print("[bold yellow]Sucessfuly saved to x_post.parquet[/bold yellow]")
# x_user confirmation
print("[bold yellow]x_user_df describe:[/bold yellow]")
print(x_user_df.describe())
print(x_user_df.shape)
print("[bold yellow]x_user_df head(5):[/bold yellow]")
print(x_user_df.head())
x_user_df.to_parquet("./data/x_user.parquet", index=False)
print("[bold yellow]Sucessfuly saved to x_user.parquet[/bold yellow]")

Processing may_july_chunk_365.csv: 100%|██████████| 50000/50000 [00:05<00:00, 9626.49it/s] 
/tmp/ipykernel_7493/3552749426.py:2: DtypeWarning: Columns (6,7,8,9,12,17,22) have mixed types. Specify dtype option on import or set low_memory=False.
  current_df = pd.read_csv(path)
Processing may_july_chunk_376.csv: 100%|██████████| 50000/50000 [00:05<00:00, 9968.83it/s] 
/tmp/ipykernel_7493/3552749426.py:2: DtypeWarning: Columns (23) have mixed types. Specify dtype option on import or set low_memory=False.
  current_df = pd.read_csv(path)
Processing may_july_chunk_363.csv: 100%|██████████| 50000/50000 [00:05<00:00, 9878.61it/s] 
/tmp/ipykernel_7493/3552749426.py:2: DtypeWarning: Columns (1,6,8,9,22) have mixed types. Specify dtype option on import or set low_memory=False.
  current_df = pd.read_csv(path)
Processing may_july_chunk_379.csv: 100%|██████████| 49999/49999 [00:04<00:00, 12430.66it/s]
/tmp/ipykernel_7493/3552749426.py:2: DtypeWarning: Columns (21,22,30) have mixed types. Specify d

x_post_df describe:

post_id       user_id                           date  \
count  1.092610e+05  1.092610e+05                         109261   
mean   1.810242e+18  9.082368e+17  2024-07-07 19:23:49.476208384   
min    1.653977e+18  5.852300e+04            2023-05-04 00:00:00   
25%    1.803308e+18  1.205104e+09            2024-06-19 00:00:00   
50%    1.812371e+18  1.207691e+18            2024-07-14 00:00:00   
75%    1.814534e+18  1.569336e+18            2024-07-20 00:00:00   
max    1.849581e+18  1.848412e+18            2024-10-24 00:00:00   
std    1.225932e+16  7.265391e+17                            NaN   

          like_count    reply_count  retweet_count    quote_count  
count  109261.000000  109261.000000  109261.000000  109261.000000  
mean       21.707984       2.029095       5.125672       0.464786  
min         0.000000       0.000000       0.000000       0.000000  
25%         0.000000       0.000000       0.000000       0.000000  
50%         0.000000       0.000000       0.000000       0.000000  
75%         1.000000       1.000000       0.000000       0.000000  
max     56883.000000    3149.000000   17204.000000    2066.000000  
std       483.466131      28.041829     100.203162      13.556495

(109261, 9)

x_post_df head(5):

...

user_id  followers_count  follows_count
count  4.757100e+04     4.757100e+04   47571.000000
mean   9.617396e+17     1.163452e+04    1479.147842
std    7.133603e+17     2.480131e+05    4670.349009
min    5.852300e+04     0.000000e+00       0.000000
25%    2.282293e+09     1.080000e+02     129.000000
50%    1.267242e+18     3.540000e+02     421.000000
75%    1.586352e+18     1.402000e+03    1294.000000
max    1.848412e+18     2.569526e+07  276711.000000

(47571, 5)

x_user_df head(5):

user_id                 name  \
0  1181549520385122304                leflu   
1  1521599300656107523  Monsieur Blacky boy   
2  1335615761365217288            Ennio Red   
3  1710461091157843975        user016777v01   
4  1499768906973593600                  FB.   

                                                 bio  followers_count  \
0            jecrit mal mais je vous aime quand meme              327   
1                                                                 269   
2  Love to triggers leftist liberals .i am Conser...            12169   
3                                                                 212   
4   Nouveau et amélioré. Toujours même mauvais goût!              501   

   follows_count  
0            291  
1            178  
2           9901  
3            146  
4            363

Sucessfuly saved to x_user.parquet